#### Task

Employees' dataset: "/mnt/training/manufacturing-org/employees/employees.csv"

Using window functions, find the **employees** who have worked in a specific **department** the *longest* and *shortest* time.

Resulting dataframe should have 3 columns: employee_name, department, employment_duration

Employment_duration should have 2 possible values: 
* **longest**
  * The employee has worked in the department for the longest time. Based on column _active_record_start_
* **shortest**
  * The employee has worked in the department for the shortest time. Based on column _active_record_start_

Dataframe should have total 6 rows.

Example df.take(3):</br>
<table>
  <tr>
    <th>employee_name</th>
    <th>department</th>
    <th>employment_duration</th>
  </tr>
  <tr>
    <td>CISNEROS JR, HERBERT</td>
    <td>OFFICE</td>
    <td>shortest</td>
  </tr>
  <tr>
    <td>CRAVEN, KEVIN J</td>
    <td>OFFICE</td>
    <td>longest</td>
  </tr>
  <tr>
    <td>WRIGHT, RONALD G</td>
    <td>PRODUCTION</td>
    <td>shortest</td>
  </tr>
</table>

Note: highest points are awarded to solutions where dataframe has least transformations.</br>
Should be doable with 2 window functions and 2 transformations.

In [0]:
# we need to import the Window API and instantiate a Window specification object
# let's also import pyspark.sql.functions for using aggregations and functions  

import pyspark.sql.functions as F
from pyspark.sql import Window

df = spark.read.csv("/mnt/training/manufacturing-org/employees/employees.csv", header=True, inferSchema=True)

window_spec = Window.partitionBy("department").orderBy("active_record_start")
window_spec_desc = Window.partitionBy("department").orderBy(F.desc("active_record_start"))

display(df
       .withColumn("oldest", F.row_number().over(window_spec)) 
       .withColumn("newest", F.row_number().over(window_spec_desc))
       .filter((F.col("newest") == 1) | (F.col("oldest") == 1))
       .selectExpr("employee_name", "department", "case when oldest = 1 then 'longest' when newest = 1 then 'shortest' else null end employment_duration")
       )
